# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [62]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import numpy as np
import os

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, read_viewing_conditions, viewing_condition_index
from visiprog.data import read_viewing_conditions, polar_to_euclidean, angle_between
from IPython.display import display, HTML

from scipy.spatial import Voronoi, SphericalVoronoi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
df_viewing = read_viewing_conditions()

np.savetxt('df_viewing.csv', df_viewing, fmt='%.4f', delimiter=',')

print(df_viewing.columns.values)
print(df_viewing.shape)

['view_theta' 'view_phi' 'illum_theta' 'illum_phi' 'illum_x' 'illum_y'
 'illum_z' 'view_x' 'view_y' 'view_z']
(205, 10)


In [25]:
def make_image_html(image_path):
     return '<img src="{}" width="100" height="100" style="display:inline;margin:1px"/>'.format(image_path)

In [36]:
groups = read_VSP_label()
list_img = read_img_list()

traces = []

group = groups[169]

for i in group: 
    viewing_index = viewing_condition_index(list_img[i])
    
    illum_x = df_viewing['illum_x'][viewing_index]
    illum_y = df_viewing['illum_y'][viewing_index]
    illum_z = df_viewing['illum_z'][viewing_index]

    view_x = df_viewing['view_x'][viewing_index]
    view_y = df_viewing['view_y'][viewing_index]
    view_z = df_viewing['view_z'][viewing_index]
    
    line_x = [illum_x, illum_x, 0, view_x]
    line_y = [illum_y - 0.05, illum_y, 0, view_y]
    line_z = [illum_z,illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name=os.path.basename(list_img[i])
    )
    
    traces.append(line_scatter)

layout = go.Layout(
    title='Angles of VSP group',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='vspgroup')

# display the images as well
display(HTML(''.join(make_image_html(list_img[i]) for i in group) + '<br>' + \
            '\t'.join(os.path.basename(list_img[i]) for i in group)))

# Formal analysis using graph and minimum spanning tree

In [33]:
groups = read_VSP_label(pappas_only=True, sorted_by_material=True)
list_img = read_img_list()

print(len(groups))

mst_distances = []

for group in groups:
    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        index_i = viewing_condition_index(list_img[group[i]])
        
        illum_x_1 = df_viewing['illum_x'][index_i]
        illum_y_1 = df_viewing['illum_y'][index_i]
        illum_z_1 = df_viewing['illum_z'][index_i]
        
        view_x_1 = df_viewing['view_x'][index_i]
        view_y_1 = df_viewing['view_y'][index_i]
        view_z_1 = df_viewing['view_z'][index_i]
        
        for j in range(i + 1, len(group)):
            index_j = viewing_condition_index(list_img[group[j]])
            illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(df_viewing['illum_theta'][index_j],df_viewing['illum_phi'][index_j])
            view_x_2, view_y_2, view_z_2 = polar_to_euclidean(df_viewing['view_theta'][index_j],df_viewing['view_phi'][index_j])
            
            weight_illum = angle_between((illum_x_1, illum_y_1, illum_z_1),(illum_x_2, illum_y_2, illum_z_2))
            weight_view = angle_between((view_x_1, view_y_1, view_z_1),(view_x_2, view_y_2, view_z_2))
        
            G.add_edge(i, j, weight=weight_illum)
            G.add_edge(i, j, weight=weight_view)
            
#     print(G.edges(data=True))
#     print("\n\nSorting the MST\n\n")
    
    # minimum spanning tree
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_distances.append(distances)

456


In [20]:
mst_out = np.asarray(mst_distances)
np.savetxt('mst_distances.csv', mst_out, fmt='%.4f', delimiter=',')
print(mst_out.shape)

(456, 8)


(604, 8)


# Formal analysis using graph and VORONOI DIAGRAM

Why? Since angle does not explain a lot. It could be better explained through spatial adjacency.

In [56]:
# building spatial graph for illumination angles
illum_points = df_viewing[['illum_theta', 'illum_phi']].as_matrix()
print(illum_points.shape)


illum_vor = Voronoi(illum_points)
print(illum_vor.ridge_points)
illum_edges = illum_vor.ridge_points

# illum_vor = SphericalVoronoi(illum_points, radius=1, center=np.array([0,0,0]))

# print(illum_vor.vertices)

(205, 2)
[[ 54  99]
 [ 54  53]
 [ 54  50]
 [ 54  52]
 [ 54  49]
 [ 54  45]
 [ 17  28]
 [ 17  22]
 [ 17  14]
 [ 17   2]
 [ 81  42]
 [ 81  37]
 [ 81  28]
 [ 81  22]
 [ 42  49]
 [ 42  37]
 [ 28  22]
 [ 28  14]
 [ 28  26]
 [ 28  37]
 [ 65  14]
 [ 65   2]
 [ 65   0]
 [ 65  26]
 [ 65  73]
 [ 65   9]
 [ 14   2]
 [ 14  26]
 [  2   0]
 [  0   1]
 [  0  12]
 [  0   9]
 [  0   3]
 [ 49  37]
 [ 49  52]
 [ 49  91]
 [135  84]
 [135  41]
 [135  45]
 [135  53]
 [135  94]
 [ 45  53]
 [ 45  50]
 [ 45  86]
 [ 45  84]
 [ 53  99]
 [ 94  99]
 [ 94  39]
 [132  50]
 [132  86]
 [132 161]
 [132  52]
 [132  95]
 [ 50  52]
 [ 50  86]
 [ 52  91]
 [ 52  95]
 [  3  12]
 [  3   1]
 [  3  15]
 [  3 145]
 [ 82  79]
 [ 82  40]
 [ 82  73]
 [ 82  95]
 [ 82  91]
 [ 82  26]
 [ 82  37]
 [ 40  79]
 [ 40 161]
 [ 40  95]
 [ 73   9]
 [ 73  20]
 [ 73  79]
 [ 73  26]
 [ 95 161]
 [ 95  91]
 [ 91  37]
 [ 26  37]
 [ 80  84]
 [ 80  41]
 [ 80  75]
 [ 80  21]
 [ 80  30]
 [ 41  39]
 [ 41  84]
 [ 21  19]
 [ 21  10]
 [ 21  13]
 [ 21  75]
 

In [47]:
sv = illum_vor
points = illum_points

%matplotlib notebook
import matplotlib.pyplot as plt

import scipy
from matplotlib import colors
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

fig = plt.figure()
ax = fig.add_subplot('111', projection='3d')
# ax.plot_surface(x, y, z, rstride=5, cstride=5, color='y', alpha=0.1)
# ax.scatter(points[:,0], points[:,1], points[:,2])

# sv.sort_vertices_of_regions()
# this is not yet completely accurate
for n in range(0, len(sv.regions)):
    region = sv.regions[n]
    ax.scatter(points[n, 0], points[n, 1], points[n, 2], c='b')
    random_color = colors.rgb2hex(scipy.rand(3))
    polygon = Poly3DCollection([sv.vertices[region]], alpha=1.0)
    polygon.set_color(random_color)
    ax.add_collection3d(polygon)

ImportError: No module named 'PyQt4'

In [57]:
traces = []

for edge in illum_edges:
    
    i1 = edge[0]
    i2 = edge[1]
    
    illum_x_1, illum_y_1, illum_z_1 = polar_to_euclidean(df_viewing['illum_theta'].iloc[i1],df_viewing['illum_phi'].iloc[i1])
    illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(df_viewing['illum_theta'].iloc[i2],df_viewing['illum_phi'].iloc[i2])
    
    line_x = [illum_x_1, illum_x_2]
    line_y = [illum_y_1, illum_y_2]
    line_z = [illum_z_1, illum_z_2]
    
    line_spatial_adjacent = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z
    )
    
    traces.append(line_spatial_adjacent)

In [58]:
layout = go.Layout(
    title='Ajacent graph',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)


fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='spatial')